In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

### Генерация признаков


#### Зачем важно создание новых признаков?

Создание **новых признаков** как **комбинация** существующих. Это отличный способ добавить **доменные знания** в набор данных.

### Загрузка данных

In [3]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]

data = pd.read_csv('./data/titanic.csv', usecols=use_cols)


In [4]:
data.head(3)

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,male,22.0,1,7.2500
1,1,1,female,38.0,1,71.2833
2,1,3,female,26.0,0,7.9250


In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, data.Survived, test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((623, 6), (268, 6))

### Производный признак на основе отсутствующих данных

Как упомянуто в разделе про пропуски, мы можем создать новый бинарный признак, обозначающий, есть ли у наблюдений отсутствующее значение в исходном признаке, со значением 0/1.

### Простой статистический производный признак

   Создание новых признаков путем выполнения простых статистических расчетов на основе исходных признаков, включая:
   - count/sum (количество/сумма)
   - average/median/mode (среднее/медиана/мода)
   - max/min (максимум/минимум)
   - stddev/variance (стандартное отклонение/дисперсия)
   - range/IQR (размах/межквартильный размах)
   - Coefficient of Variation (коэффициент вариации)
   - time span/interval (промежуток времени/интервал)

   Возьмем, например, журнал звонков: мы можем создать новые признаки, такие как: количество звонков, количество входящих/исходящих звонков, средняя длительность звонков, средняя длительность звонков в месяц, максимальная длительность звонков и так далее.

### Пересечение признаков

   После получения нескольких простых статистических производных признаков, мы можем объединить их вместе. Общие измерения, используемые для пересечения, включают:
   - время
   - регион
   - типы бизнеса

   Продолжим использовать журнал звонков в качестве примера: мы можем создать пересекающиеся признаки, такие как: количество звонков в ночное/дневное время, количество звонков в различных типах бизнеса (банки/такси/путешествия/гостиничный бизнес), количество звонков за последние 3 месяца и многое другое. Многие из статистических расчетов, упомянутых в предыдущем пункте, могут быть использованы снова для создания дополнительных признаков.


### Отношения и пропорции

Для прогнозирования будущей производительности продаж кредитных карт в отделении, отношения, такие как продажи по кредитным картам / продавец или продажи по кредитным картам / маркетинговые расходы, будут более мощными, чем просто использование абсолютного числа проданных карт в отделении.

### Произведения категориальных признаков

 Предположим, у нас есть категориальный признак A с двумя возможными значениями {A1, A2}. Пусть B будет признаком с вариантами {B1, B2}. Тогда перекрестный признак между A и B примет одно из следующих значений: {(A1, B1), (A1, B2), (A2, B1), (A2, B2)}. Вы можете дать этим "комбинациям" любые имена, которые вам нравятся. Просто помните, что каждая комбинация обозначает синергию между информацией, содержащейся в соответствующих значениях A и B.

### Полиномиальное расширение

 Произведение признаков также может применяться к числовым признакам, что приводит к созданию нового взаимодействующего признака между A и B. Это можно легко сделать с помощью `PolynomialFeatures` из библиотеки sklearn, которая генерирует новый набор признаков, состоящий из всех полиномиальных комбинаций признаков с степенью, меньшей или равной заданной степени. Например, три исходных признака {X1, X2, X3} могут создать набор признаков {1, X1X2, X1X3, X2X3, X1X2X3} со степенью 2.


In [6]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=2,include_bias=False).fit(X_train[['Pclass','SibSp']])
tmp = pf.transform(X_train[['Pclass','SibSp']])
X_train_copy = pd.DataFrame(tmp,columns=pf.get_feature_names(['Pclass','SibSp']))
print(X_train_copy.head(6))

   Pclass  SibSp  Pclass^2  Pclass SibSp  SibSp^2
0     1.0    0.0       1.0           0.0      0.0
1     1.0    1.0       1.0           1.0      1.0
2     3.0    5.0       9.0          15.0     25.0
3     1.0    0.0       1.0           0.0      0.0
4     3.0    1.0       9.0           3.0      1.0
5     2.0    1.0       4.0           2.0      1.0


## Feature Learning by Trees


В алгоритмах, основанных на деревьях, каждому образцу будет назначен определенный листовой узел. Путь принятия решения к каждому узлу можно рассматривать как новый нелинейный признак, и мы можем создать N новых бинарных признаков, где n равно общему числу листовых узлов в дереве или ансамбле деревьев. Затем признаки можно передавать в другие алгоритмы, такие как логистическая регрессия.
   
  Идея использования алгоритма дерева для создания новых признаков была впервые представлена Facebook в этой [статье](http://quinonero.net/Publications/predicting-clicks-facebook.pdf).
   
   Хорошей чертой этого метода является то, что мы можем получить сложные комбинации нескольких признаков, что информативно (как создано алгоритмом обучения дерева). Это позволяет сэкономить нам много времени по сравнению с ручным созданием пересекающихся признаков и широко используется в CTR (кликабельность) в индустрии онлайн-рекламы.


In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder

ensemble = GradientBoostingClassifier(n_estimators=20)
one_hot = OneHotEncoder()

X_train = X_train[[ 'Pclass', 'Age', 'Fare', 'SibSp']].fillna(0)
X_test = X_test[[ 'Pclass', 'Age', 'Fare', 'SibSp']].fillna(0)

ensemble.fit(X_train, y_train)

y_pred = ensemble.predict_proba(X_test)[:, 1]

print("ROC-AUC GBDT score：", roc_auc_score(y_test, y_pred))

ROC-AUC GBDT score： 0.7624702380952381


In [9]:
X_leaf_index

array([[ 7.,  7.,  6., ...,  4.,  7.,  4.],
       [ 7.,  7.,  6., ..., 14.,  7.,  7.],
       [11., 11., 11., ...,  4.,  6., 11.],
       ...,
       [10., 10., 10., ...,  4.,  6., 10.],
       [13., 14., 13., ...,  4.,  7., 13.],
       [ 7.,  7.,  6., ...,  6.,  7.,  7.]])

In [8]:
X_leaf_index = ensemble.apply(X_train)[:, :, 0] # получаем список индексов листьев дерева
one_hot.fit(X_leaf_index)   
X_one_hot = one_hot.transform(X_leaf_index)  

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_one_hot,y_train)
y_pred = lr.predict_proba(
    one_hot.transform(ensemble.apply(X_test)[:, :, 0]))[:, 1]
print("ROC-AUC GBDT derived feature + LR：", roc_auc_score(y_test, y_pred))

ROC-AUC GBDT derived feature + LR： 0.7746130952380953


### Обучение признаков с использованием глубоких нейронных сетей

Cоздание признаков вручную требует много усилий и не всегда гарантирует хороший результат, особенно когда у нас много признаков для работы. Обучение признаков с использованием деревьев можно рассматривать как первую попытку создания признаков автоматически. Нейронные сети и подходы, таких как автоэнкодеры и Transfer Learning, демонстрируют успехи в извелечении признаков. Однако такие признаки имеют ограниченную интерпретируемость, и для извлечения качественных результатов нейронной сетью требуется гораздо больше данных.